In [ ]:
""" Artificial Bee Colony Algorithm

Input
- D: number of decision variables
- S: swarm size
- T: number of cycles
- limit: decides when scouts phase needs to be executed, often taken Np*D

Np: number of food sources/employed bees/onlooker bees


Output


"""

function ArtificialBeeColonization(D,S, T, limit)
    Np = S/2
    population = initialize_population(D, bounds_lower, bounds_upper, Np)
    objective_values = compute_objective(population, false)
    fitness_values = compute_fitness(objective_values)
    
     

    return penalty
end	
